In [20]:
import whisper
loaded_model = whisper.load_model("medium") # 9sec    
from pytube import YouTube

import sys, os

sys.path.append(os.path.dirname(os.path.abspath('')))
from utils import get_transcript_list, format_timestamp, write_srt_ko_v3, logging_time

@logging_time
def prepare(link, loaded_model, save_path):
    yt = YouTube(link)
    path = yt.streams.filter(only_audio=True)[0].download(filename=save_path+"audio.mp3")
    options = dict(task="transcribe", best_of=5)
    results = loaded_model.transcribe(path, **options)
    
    return results

@logging_time
def download_video(link, save_path):
    yt = YouTube(link)
    video = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(output_path=save_path)
    return video

# link = "https://www.youtube.com/watch?v=1aA1WGON49E" # 1분 20초
link = "https://www.youtube.com/watch?v=5m-5dMP0NTI" # 5분
# link = "https://www.youtube.com/watch?v=LK5j3pp0Too&t=2s" # 16분

results = prepare(link, loaded_model, '.')

download_video(link, '.')

걸린 시간[prepare]: 33.8286554813385 sec
걸린 시간[download_video]: 2.158722400665283 sec


'/workspace/whisper_auto_subtitle_generator/parse_back/test/./Learn English with Subtitles - 5 Minute Daily News June 30 2019 - Updated Daily.mp4'

In [21]:
# Texted 1

import google.generativeai as genai
import os

from io import StringIO
from typing import Iterator, TextIO
import sys
import re

sys.path.append(os.path.dirname(os.path.abspath('')))
from utils import get_transcript_list, format_timestamp, write_srt_ko_v3, logging_time

class Translator_GoogleGemini_Multi_Separate:
    def __init__(self):
        genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))
        self.translator = genai.GenerativeModel('gemini-pro')

    def translate_one(self, text):
        prompt = f"""

        # Rules:
        - Since the translation will be used for subtitles, you must strictly follow the output format.
        - Numbering must be strictly adhered to. If the response does not have the same number of lines as the numbering in the input, You must adjust the output to match the numbering format, even if it means breaking up sentences across multiple lines.
        - Each line must have a text or contents.
        
        # Example:
        - Input1: <paragraph>I'm at 44 seconds right now. That means we've got time for one final joke. <lb/>Because they're the ones who get it seen and get it shared. <lb/> You see, back in 2009, we all had these weird little things called attention spans. </paragraph>
        - Output1: 지금은 44초입니다. 즉, 마지막 농담 하나 할 시간이 있습니다. <lb/> 본 사람들이 보고 공유하기 때문입니다. <lb/> 아시겠지만, 2009년에는 저희 모두가 주의력이라는 이상한 작은 것을 가지고 있었습니다.

        - Input2: <paragraph>Sudan's main opposition group says heavily armed security forces raided its offices <lb/>and blocked a press conference on the eve of Sunday's protests against military rule. <lb/>The Sudanese Professional Association had called for a news conference to unveil plans for the rally,</paragraph>
        - Output2: 수단의 주요 야권 단체는 중무장한 군대가 자신들의 사무실을 급습했다고 말했습니다. <lb/> 그리고 군부 통치에 반대하는 일요일 시위 전날 언론 기자회견을 막았습니다. <lb/> 수단 전문가 협회는 집회를 위한 계획을 발표하기 위해 기자회견을 소집했습니다.

        - Input3: <paragraph>The sun was shining brightly in the clear blue sky.<lb/> Birds were chirping happily, welcoming the new day.<lb/> A gentle breeze carried the sweet scent of blooming flowers.</paragraph>
        - Output3: 태양은 맑은 하늘에 환하게 빛나고 있었다.<lb/> 새들은 행복하게 지저귀며 새로운 하루를 맞이했다.<lb/> 부드러운 바람이 피어난 꽃들의 향기를 실어 나르고 있었다.
        
        # Original Paragraph: 
        <paragraph>{text}</paragraph>
        
        # Your translation:"""
        
        response = self.translator.generate_content(
                        prompt,
                        generation_config=genai.types.GenerationConfig(
                            candidate_count=1,
                            temperature=0.4,
                        ),
                    )
        
        translated_text_list = response.text.split("<lb/>")
        translated_text_list = self.remove_strips(translated_text_list)        
        return translated_text_list[0]
        
    def translate(self, indice, text_list: list, source_lang="English", target_lang="Korean"):
        result = ""
        for idx, text in enumerate(text_list, start=indice+1):
            result += f"{idx}. {text}<lb/>"
            
        prompt = f"""

        # Rules:
        - Since the translation will be used for subtitles, you must strictly follow the output format.
        - Numbering must be strictly adhered to. If the response does not have the same number of lines as the numbering in the input, You must adjust the output to match the numbering format, even if it means breaking up sentences across multiple lines.
        - Each line must have a text or contents.
        
        # Example:
        - Input1: <paragraph>I'm at 44 seconds right now. That means we've got time for one final joke. <lb/>Because they're the ones who get it seen and get it shared. <lb/> You see, back in 2009, we all had these weird little things called attention spans. </paragraph>
        - Output1: 지금은 44초입니다. 즉, 마지막 농담 하나 할 시간이 있습니다. <lb/> 본 사람들이 보고 공유하기 때문입니다. <lb/> 아시겠지만, 2009년에는 저희 모두가 주의력이라는 이상한 작은 것을 가지고 있었습니다.

        - Input2: <paragraph>Sudan's main opposition group says heavily armed security forces raided its offices <lb/>and blocked a press conference on the eve of Sunday's protests against military rule. <lb/>The Sudanese Professional Association had called for a news conference to unveil plans for the rally,</paragraph>
        - Output2: 수단의 주요 야권 단체는 중무장한 군대가 자신들의 사무실을 급습했다고 말했습니다. <lb/> 그리고 군부 통치에 반대하는 일요일 시위 전날 언론 기자회견을 막았습니다. <lb/> 수단 전문가 협회는 집회를 위한 계획을 발표하기 위해 기자회견을 소집했습니다.

        - Input3: <paragraph>The sun was shining brightly in the clear blue sky.<lb/> Birds were chirping happily, welcoming the new day.<lb/> A gentle breeze carried the sweet scent of blooming flowers.</paragraph>
        - Output3: 태양은 맑은 하늘에 환하게 빛나고 있었다.<lb/> 새들은 행복하게 지저귀며 새로운 하루를 맞이했다.<lb/> 부드러운 바람이 피어난 꽃들의 향기를 실어 나르고 있었다.
        
        # Original Paragraph: 
        <paragraph>{result}</paragraph>
        
        # Your translation:"""

        # Generate the text response using the model
        
        
        # 최대 5번까지 재요청
        count = 0
        while count < 5: 
            
            response = self.translator.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    candidate_count=1,
                    temperature=0.4,
                ),
            )
            
            translated_text_list = response.text.split("<lb/>")
            translated_text_list = self.remove_strips(translated_text_list)
            
            count += 1
            if self.vaild_response(translated_text_list, len(text_list)):
                print("예외처리 X")
                break
            
        # 5번까지 요청했는데 안됀다 그러면 한줄한줄 번역해야함
        else:
            print("예외처리 O")
            translated_text_list = []
            
            for text in text_list:
                translated_text_list.append(self.translate_one(text))
        
        
        # return translated text_list -> list
        return translated_text_list   
    
    # 응답이 자막에 사용할 수 있는지 아닌지
    def vaild_response(self, translated_text_list, indice):
        
        # Get the translated text from the response
        
        if len(translated_text_list) > indice:
            return False
        elif len(translated_text_list) == indice:
            return True
        elif len(translated_text_list) < indice:
            return False
        
    def remove_strips(self, translated_text_list):
        
        # 각 문장의 \n과 넘버링을 제거하고
        # 제거한 리스트에서 공백 제거
        
        cleaned_list = list(filter(str.strip, [re.sub(r'^\d+\.\s*', '', item.replace('\n', '')) for item in translated_text_list]))
        
        return cleaned_list
        

def write_srt_ko_v3(transcript: Iterator[dict], file: TextIO, maxLineWidth=None):

    translator = Translator_GoogleGemini_Multi_Separate()
    
    indice = 0
    increment = 10
    
    while indice <= len(transcript):

        english_list = get_transcript_list(transcript[indice:indice+increment])
        translated_text = translator.translate(indice, english_list) 
        print("")   
        translated_text = [item.replace('<paragraph>', '').replace('</paragraph>', '') for item in translated_text]
        try:
            for i, segment in enumerate(transcript[indice:indice+increment], start=indice+1):

                print(
                    f"{i}\n"
                    f"{format_timestamp(segment['start'], always_include_hours=True, fractionalSeperator=',')} --> "
                    f"{format_timestamp(segment['end'], always_include_hours=True, fractionalSeperator=',')}\n"
                    f"{translated_text[i-indice-1]}\n",
                    file=file,
                    flush=True,
                )
        except:
            pass        
            
        indice += increment
        
@logging_time
def getSubs(segments: Iterator[dict], format: str, maxLineWidth: int) -> str:
    segmentStream = StringIO()

    if format == 'srt_ko':
        write_srt_ko_v3(segments, file=segmentStream, maxLineWidth=maxLineWidth)
    else:
        raise Exception("Unknown format " + format)

    segmentStream.seek(0)
    return segmentStream.read()

srt_ko = getSubs(results["segments"], "srt_ko", None)

with open("transcript_ko.srt", "w+",encoding='utf8') as f:
    f.writelines(srt_ko)
    f.close()

10it [00:00, 64231.30it/s]


예외처리 X



10it [00:00, 76678.32it/s]


예외처리 X



10it [00:00, 94254.02it/s]


...

10it [00:00, 85948.85it/s]


예외처리 X



1it [00:00, 11915.64it/s]


예외처리 X

걸린 시간[getSubs]: 46.92539596557617 sec


: 

In [ ]:
# Texted 2

import google.generativeai as genai
import os

from io import StringIO
from typing import Iterator, TextIO
import sys

sys.path.append(os.path.dirname(os.path.abspath('')))
from utils import get_transcript_list, format_timestamp, write_srt_ko_v3

class Translator_GoogleGemini_Multi_Separate:
    def __init__(self):
        genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))
        self.translator = genai.GenerativeModel('gemini-pro')
        
    def translate(self, indice, text_list: list, source_lang="English", target_lang="Korean", ):
        result = ""
        for idx, text in enumerate(text_list, start=1):
            result += f"{idx}. {text} \n"
            
            
        prompt = f"""Here is the prompt with appropriate instructions:

        Instructions: Translate the following sentences into Korean line-by-line, strictly maintaining the same number of sentences as the input. Clearly mark the end of each sentence output with an <eos> token. If the number of sentences in the output does not match the input, you will be shown an error and asked to regenerate the output.

        **MUST**
        - these translations will be used to **Subtitles**
        - So Each line have to contain texts or content
        - The translation can be odd or out of order, so you must strictly follow the numbering, otherwise you will be penalized.
        
        Failure to adhere to these instructions will result in an error:

        - Combining multiple input sentences into one output sentence
        - Outputting a different number of sentences than the input
        - Not including <eos> tokens at the end of each sentence
        
        Example Input:

        1. This is the first sentence.
        2. Here is the second sentence.
        ...
        ...
        20. Here is the twenty sentency.
        
        Example Correct Output:

        1. 이것은 첫 번째 문장입니다.<eos>
        2. 여기 두 번째 문장입니다.<eos>
        ...
        ...
        20. 여기 스무 번째 문장입니다.<eos>
        
        {result}

        """

        # Generate the text response using the model
        response = self.translator.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                candidate_count=1,
                temperature=0.4,
            ),
        )

        # Get the translated text from the response
        # translated_text_list = response.text.split("<lb/>")
        translated_text_list = response.text.split("<eos>")

        # return translated text_list -> list
        return translated_text_list   
    

def write_srt_ko_v3(transcript: Iterator[dict], file: TextIO, maxLineWidth=None):

    translator = Translator_GoogleGemini_Multi_Separate()
    
    indice = 0
    increment = 20
    
    while indice <= len(transcript):

        english_list = get_transcript_list(transcript[indice:indice+increment])
        translated_text = translator.translate(indice, english_list) 
        print("")   
        translated_text = [item.replace('<paragraph>', '').replace('</paragraph>', '') for item in translated_text]
        try:
            for i, segment in enumerate(transcript[indice:indice+increment], start=indice+1):

                print(
                    f"{i}\n"
                    f"{format_timestamp(segment['start'], always_include_hours=True, fractionalSeperator=',')} --> "
                    f"{format_timestamp(segment['end'], always_include_hours=True, fractionalSeperator=',')}\n"
                    f"{translated_text[i-indice-1]}\n",
                    file=file,
                    flush=True,
                )
        except:
            pass        
            
        indice += increment
        
def getSubs(segments: Iterator[dict], format: str, maxLineWidth: int) -> str:
    segmentStream = StringIO()

    if format == 'srt_ko':
        write_srt_ko_v3(segments, file=segmentStream, maxLineWidth=maxLineWidth)
    else:
        raise Exception("Unknown format " + format)

    segmentStream.seek(0)
    return segmentStream.read()

srt_ko = getSubs(results["segments"], "srt_ko", None)


In [ ]:
# Texted 3

import google.generativeai as genai
import os

from io import StringIO
from typing import Iterator, TextIO
import sys

sys.path.append(os.path.dirname(os.path.abspath('')))
from utils import get_transcript_list, format_timestamp, write_srt_ko_v3

class Translator_GoogleGemini_Multi_Separate:
    def __init__(self):
        genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))
        self.translator = genai.GenerativeModel('gemini-pro')
        
    def translate(self, indice, text_list: list, source_lang="English", target_lang="Korean", ):
        result = ""
        for idx, text in enumerate(text_list, start=indice+1):
            result += f"{idx}. {text}<lb/>"
            
        prompt = f"""

        # Rules:
        - Since the translation will be used for subtitles, you must strictly follow the output format.
        - Numbering must be strictly adhered to. If the response does not have the same number of lines as the numbering in the input, You must adjust the output to match the numbering format, even if it means breaking up sentences across multiple lines.
        - Each line must have a text or contents.
        
        # Example:
        - Input1: <paragraph>I'm at 44 seconds right now. That means we've got time for one final joke. <lb/>Because they're the ones who get it seen and get it shared. <lb/> You see, back in 2009, we all had these weird little things called attention spans. </paragraph>
        - Output1: 지금은 44초입니다. 즉, 마지막 농담 하나 할 시간이 있습니다. <lb/> 본 사람들이 보고 공유하기 때문입니다. <lb/> 아시겠지만, 2009년에는 저희 모두가 주의력이라는 이상한 작은 것을 가지고 있었습니다.

        - Input2: <paragraph>Sudan's main opposition group says heavily armed security forces raided its offices <lb/>and blocked a press conference on the eve of Sunday's protests against military rule. <lb/>The Sudanese Professional Association had called for a news conference to unveil plans for the rally,</paragraph>
        - Output2: 수단의 주요 야권 단체는 중무장한 군대가 자신들의 사무실을 급습했다고 말했습니다. <lb/> 그리고 군부 통치에 반대하는 일요일 시위 전날 언론 기자회견을 막았습니다. <lb/> 수단 전문가 협회는 집회를 위한 계획을 발표하기 위해 기자회견을 소집했습니다.

        - Input3: <paragraph>The sun was shining brightly in the clear blue sky.<lb/> Birds were chirping happily, welcoming the new day.<lb/> A gentle breeze carried the sweet scent of blooming flowers.</paragraph>
        - Output3: 태양은 맑은 하늘에 환하게 빛나고 있었다.<lb/> 새들은 행복하게 지저귀며 새로운 하루를 맞이했다.<lb/> 부드러운 바람이 피어난 꽃들의 향기를 실어 나르고 있었다.
        
        # Original Paragraph: 
        <paragraph>{result}</paragraph>
        
        # Your translation:"""

        # Generate the text response using the model
        response = self.translator.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                candidate_count=1,
                temperature=0.4,
            ),
        )

        # Get the translated text from the response
        translated_text_list = response.text.split("<lb/>")

        # return translated text_list -> list
        return translated_text_list   
    

def write_srt_ko_v3(transcript: Iterator[dict], file: TextIO, maxLineWidth=None):

    translator = Translator_GoogleGemini_Multi_Separate()
    
    indice = 0
    increment = 20
    
    while indice <= len(transcript):

        english_list = get_transcript_list(transcript[indice:indice+increment])
        translated_text = translator.translate(indice, english_list) 
        print("")   
        translated_text = [item.replace('<paragraph>', '').replace('</paragraph>', '') for item in translated_text]
        
        for i, segment in enumerate(transcript[indice:indice+increment], start=indice+1):

            print(
                f"{i}\n"
                f"{format_timestamp(segment['start'], always_include_hours=True, fractionalSeperator=',')} --> "
                f"{format_timestamp(segment['end'], always_include_hours=True, fractionalSeperator=',')}\n"
                f"{translated_text[i-indice-1]}\n",
                file=file,
                flush=True,
            )        
            
        indice += increment
        
def getSubs(segments: Iterator[dict], format: str, maxLineWidth: int) -> str:
    segmentStream = StringIO()

    if format == 'srt_ko':
        write_srt_ko_v3(segments, file=segmentStream, maxLineWidth=maxLineWidth)
    else:
        raise Exception("Unknown format " + format)

    segmentStream.seek(0)
    return segmentStream.read()

srt_ko = getSubs(results["segments"], "srt_ko", None)


In [19]:
# Texted 4

import google.generativeai as genai
import os

from io import StringIO
from typing import Iterator, TextIO
import sys

sys.path.append(os.path.dirname(os.path.abspath('')))
from utils import get_transcript_list, format_timestamp, write_srt_ko_v3, write_srt

class Translator_GoogleGemini_Multi_Separate:
    def __init__(self):
        genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))
        self.translator = genai.GenerativeModel('gemini-pro')
        
    def translate(self, indice, text_list: list, source_lang="English", target_lang="Korean"):
                   
        prompt = f"""Translate english into korean

        # Rules:
        - Since the translation will be used for subtitles, you must strictly follow the output format.
        - Numbering must be strictly adhered to. If the response does not have the same number of lines as the numbering in the input, You must adjust the output to match the numbering format, even if it means breaking up sentences across multiple lines.
        - Each line must have a text or contents.

        
        # Original Paragraph: 
        <paragraph>{text_list}</paragraph>
        
        # Your translation:"""

        # Generate the text response using the model
        response = self.translator.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                candidate_count=1,
                temperature=0.4,
            ),
        )

        # Get the translated text from the response
        translated_text_list = response.text.split("\n")

        # return translated text_list -> list
        return translated_text_list   
    

def write_srt_ko_v3(transcript: Iterator[dict], file: TextIO, maxLineWidth=None):

    translator = Translator_GoogleGemini_Multi_Separate()
    
    indice = 0
    increment = 40
    
    while indice <= len(transcript):

        with open('transcript.srt', 'r', encoding='utf-8') as f:
            lines = f.readlines()
            
            english_list = lines[indice:indice+increment]
            translated_text = translator.translate(indice, english_list) 
            print("")   
            translated_text = [item.replace('<paragraph>', '').replace('</paragraph>', '') for item in translated_text]
            
            for i, segment in enumerate(lines[indice:indice+increment], start=indice+1):
                try:
                    print(
                        f"{i}\n"
                        f"{format_timestamp(segment['start'], always_include_hours=True, fractionalSeperator=',')} --> "
                        f"{format_timestamp(segment['end'], always_include_hours=True, fractionalSeperator=',')}\n"
                        f"{translated_text[i-indice-1]}\n",
                        file=file,
                        flush=True,
                    )   
                except:
                    pass     
                
            indice += increment
        
def getSubs(segments: Iterator[dict], format: str, maxLineWidth: int) -> str:
    segmentStream = StringIO()

    if format == 'srt':
        write_srt(segments, file=segmentStream, maxLineWidth=maxLineWidth)
    elif format == 'srt_ko':
        write_srt_ko_v3(segments, file=segmentStream, maxLineWidth=maxLineWidth)
    else:
        raise Exception("Unknown format " + format)

    segmentStream.seek(0)
    return segmentStream.read()

srt = getSubs(results["segments"], "srt", None)
with open("./test/transcript_en.srt", "w+",encoding='utf8') as f:
    f.writelines(srt)
    f.close()
    
    
srt_ko = getSubs(results["segments"], "srt_ko", None)


FileNotFoundError: [Errno 2] No such file or directory: 'test/transcript_en.srt'